# About This Notebook

This implementation is based on a vanilla Efficientnet_B0 in Pytorch for the Pawpularity Competition.  
This model uses **both images and dense features** for score prediction.  
This scores around [insert here].

Training Params: -
1. **Dataset**: - 3-channel RGB Images (256x256) with separate dense features
2. **Augmentations**: - Resize, Normalize, HorizontalFlip, VerticalFlip, RandomBrightness, RandomResizedCrop, HueSaturationValue, RandomBrightnessContrast
3. **Optimizer**: - AdamW
4. **Scheduler**: - CosineAnnealingLR
5. **Model**: - Efficientnet-B0
6. **Initial Weights**: - Imagenet
5. **Max Epochs**: - 20 (~1 min per epoch on Tesla T4 GPU)
6. **Saved Weights**: - 5-fold ensemble. Weights having highest OOF score on RMSE metric were saved.

**If you find this notebook useful and use parts of it in your work, please don't forget to show your appreciation by upvoting this kernel. That keeps me motivated and inspires me to write and share these public kernels. 😊**

# Get GPU Info

In [ ]:
!nvidia-smi

# Import

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [ ]:
# Asthetics
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# General
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import os
import glob
import random
import cv2
pd.set_option('display.max_columns', None)

# Image Aug
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

# Deep Learning
import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Random Seed Initialize
RANDOM_SEED = 42

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

# Device Optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

In [ ]:
csv_dir = '../input/petfinder-pawpularity-score'
test_dir = '../input/petfinder-pawpularity-score/test'
models_dir = '../input/pawpularity-contest-models/efficientnet_b0'

test_file_path = os.path.join(csv_dir, 'test.csv')
sample_sub_file_path = os.path.join(csv_dir, 'sample_submission.csv')
print(f'Test file: {test_file_path}')
print(f'Models path: {models_dir}')

In [ ]:
test_df = pd.read_csv(test_file_path)
sample_df = pd.read_csv(sample_sub_file_path)

In [ ]:
def return_filpath(name, folder):
    path = os.path.join(folder, f'{name}.jpg')
    return path

In [ ]:
test_df['image_path'] = test_df['Id'].apply(lambda x: return_filpath(x, folder=test_dir))

In [ ]:
test_df.head()

# CFG

In [ ]:
params = {
    'model': 'efficientnet_b0',
    'dense_features': ['Subject Focus', 'Eyes', 'Face', 'Near',
                       'Action', 'Accessory', 'Group', 'Collage',
                       'Human', 'Occlusion', 'Info', 'Blur'],
    'pretrained': False,
    'inp_channels': 3,
    'im_size': 256,
    'device': device,
    'batch_size': 32,
    'num_workers' : 2,
    'out_features': 1,
    'debug': False
}

In [ ]:
if params['debug']:
    test_df = test_df.sample(frac=0.1)

# Augmentations

In [ ]:
def get_test_transforms(DIM = params['im_size']):
    return albumentations.Compose(
        [
          albumentations.Resize(DIM,DIM),
          albumentations.Normalize(
              mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225],
              ),
          ToTensorV2(p=1.0)
        ]
    )

# Dataset

In [ ]:
class CuteDataset(Dataset):
    def __init__(self, images_filepaths, dense_features, targets, transform=None):
        self.images_filepaths = images_filepaths
        self.dense_features = dense_features
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        dense = self.dense_features[idx, :]
        label = torch.tensor(self.targets[idx]).float()
        return image, dense, label

# CNN Model

In [ ]:
class PetNet(nn.Module):
    def __init__(self, model_name=params['model'], out_features=params['out_features'], inp_channels=params['inp_channels'],
                 pretrained=params['pretrained'], num_dense=len(params['dense_features'])):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels)
        if model_name.split('_')[0] == 'efficientnet':
            out_channels = self.model.conv_stem.out_channels
            kernel_size = self.model.conv_stem.kernel_size
            stride = self.model.conv_stem.stride
            padding = self.model.conv_stem.padding
            bias = self.model.conv_stem.bias
            self.model.conv_stem = nn.Conv2d(inp_channels, out_channels,
                                             kernel_size=kernel_size, stride=stride,
                                             padding=padding, bias=bias)
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Identity()
        elif model_name.split('_')[0] == 'nfnet':
            n_features = self.model.head.fc.in_features
            self.model.head.fc = nn.Identity()
        elif model_name in ['resnet18d', 'resnet50d', 'resnet152d',
                            'seresnet50', 'seresnext26d_32x4d', 'seresnext50_32x4d',
                            'resnetblur18', 'resnetblur50']:
            n_features = self.model.fc.in_features
            self.model.fc = nn.Identity()

        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(n_features + num_dense, out_features)
    
    def forward(self, image, dense):
        embeddings = self.model(image)
        x = self.dropout(embeddings)
        x = torch.cat([x, dense], dim=1)
        output = self.fc(x)
        return output

# Prediction

In [ ]:
predicted_labels = None
for model_name in glob.glob(models_dir + '/*.pth'):
    model = PetNet()
    model.load_state_dict(torch.load(model_name))
    model = model.to(params['device'])
    model.eval()

    test_dataset = CuteDataset(
        images_filepaths = test_df['image_path'].values,
        dense_features = test_df[params['dense_features']].values,
        targets = sample_df['Pawpularity'].values,
        transform = get_test_transforms()
    )
    test_loader = DataLoader(
        test_dataset, batch_size=params['batch_size'],
        shuffle=False, num_workers=params['num_workers'],
        pin_memory=True
    )

    temp_preds = None
    with torch.no_grad():
        for (images, dense, target) in tqdm(test_loader, desc=f'Predicting. '):
            images = images.to(params['device'], non_blocking=True)
            dense = dense.to(params['device'], non_blocking=True)
            predictions = model(images, dense).to('cpu').numpy()
            
            if temp_preds is None:
                temp_preds = predictions
            else:
                temp_preds = np.vstack((temp_preds, predictions))

    if predicted_labels is None:
        predicted_labels = temp_preds
    else:
        predicted_labels += temp_preds
        
predicted_labels /= (len(glob.glob(models_dir + '/*.pth')))

In [ ]:
sub_df = pd.DataFrame()
sub_df['Id'] = test_df['Id']
sub_df['Pawpularity'] = predicted_labels

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv', index=False)

**If you find this notebook useful and use parts of it in your work, please don't forget to show your appreciation by upvoting this kernel. That keeps me motivated and inspires me to write and share these public kernels. 😊**